<h1><center>Classify Review Sentiments</center></h1>
<h2><center>Embeddings</center></h2>
<h3><center>Build AI Apps - Beginner Level</center></h3>

## Before you start

In order to complete the project you will need to create a developer account with OpenAI and store your API key as an environment variable. Instructions for these steps are outlined below.

### Create a developer account with OpenAI

1. Go to the [API signup page](https://platform.openai.com/signup). 

2. Create your account (you'll need to provide your email address and your phone number).

3. Go to the [API keys page](https://platform.openai.com/account/api-keys). 

4. Create a new secret key.


5. **Take a copy of it**. (If you lose it, delete the key and create a new one.)

### Add a payment method

OpenAI sometimes provides free credits for the API, but this can vary based on geography. You may need to add debit/credit card details. 

**Using the `gpt-3.5-turbo` model in this project should incur a cost less than 1 US cent (but if you rerun tasks, you will be charged every time).** For more information on pricing, see [OpenAI's pricing page](https://openai.com/pricing).

1. Go to the [Payment Methods page](https://platform.openai.com/account/billing/payment-methods).

2. Click Add payment method.

3. Fill in your card details.

### Install open ai library

In [81]:
# !pip install openai

### Load Open AI Key

In [82]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(find_dotenv())

OPEN_API_KEY=os.getenv('OPENAI_API_KEY')

# print(OPEN_API_KEY)


### Create Client

In [83]:

from openai import OpenAI

# Define Model
# model="gpt-4o-mini"
emb_model="text-embedding-3-small"

#define client
client = OpenAI(api_key=OPEN_API_KEY)

## Make Your First Call

In [84]:
# response=client.chat.completions.create(model=model,
#                               messages=[{"role":"user","content":"Tell me more about gravity like I am a 5 year old kid in 2 lines"}
#                                        ]
#                               )

# print(response.choices[0].message.content)

## Load Data

In [85]:
import pandas as pd

df = pd.read_csv("restaurant_review.csv")


In [86]:
df.head()

,Review
0,I've been to this place about two times and i ...
1,I have visited this pub last weekend with grou...
2,Surely one of the best lasagna you can have. T...
3,"Closed ambiance, loud. Food was filling, we or..."
4,It was great experience . You will find crowd ...


In [87]:
review_list = df['Review'].tolist()
top_5 = review_list[:5]

In [88]:
top_5

["I've been to this place about two times and i really liked the ambience, interior will give you feels of london streets.\ntalking about food, only good thing was lasagna other than that we ordered veg Platter and lasagna rolls which were really bad in taste. dj was not too good",
 'I have visited this pub last weekend with group of friends, we had brewed beers, veg & non-veg starters. Food tastes good, prices are reasonable, ambience is decent, service is good even though it’s on Saturday night.\n\nOverall i liked the place, I will visit again!!',
 'Surely one of the best lasagna you can have. The place never disappoints you even when you are just trying something for the first time here. Chilli cheese toast is something you have to try .\n\nGreat food at budget price with good service in a lovely ambience.\nWhat more can you ask for....',
 "Closed ambiance, loud. Food was filling, we ordered veg platter and the pizza is a must try, thin crust with a good amount of cheese, if you're 

## Generate Embeddings for Reviews



In [89]:
# response=client.embeddings.create(model=emb_model,input=top_5)
# response_dict=response.model_dump()
# len(response_dict['data'][0]['embedding'])

In [90]:
def create_embeddings(texts):
    response=client.embeddings.create(model=emb_model,input=texts)
    response_dict=response.model_dump()
    return[data['embedding'] for data in response_dict['data']]

In [91]:
review_embeddings=create_embeddings(top_5)
len(review_embeddings[0])

1536

In [92]:
sentiments = [{'label': 'Positive'}, {'label': 'Neutral'}, {'label': 'Negative'}]

In [93]:
# class_descriptions = [sentiment['label'] for sentiment in sentiments]
class_descriptions = [
    'This is a positive review expressing satisfaction and happiness.',
    'This is a neutral review with no strong feelings.',
    'This is a negative review expressing dissatisfaction and disappointment.'
]

In [94]:
class_embeddings=create_embeddings(class_descriptions)

## Find Closest Sentiment Using Cosine Distance

In [95]:
from scipy.spatial import distance

def find_closest(query_vector,embeddings):
    distances=[]
    for index, embedding in enumerate(embeddings):
        dist=distance.cosine(query_vector,embedding)
        distances.append({"distance":dist,"index":index})
    return min(distances,key=lambda x: x['distance'])

## Classify Each Review

In [99]:
result=[]
for index, review in enumerate(top_5):
    closest=find_closest(review_embeddings[index],class_embeddings)
    # print(closest)
    label=sentiments[closest['index']]['label']
    # print(label)
    result.append({'Review':review,'predicted_sentiment':label})

In [102]:
result_df=pd.DataFrame(result)
result_df

,Review,predicted_sentiment
0,I've been to this place about two times and i ...,Negative
1,I have visited this pub last weekend with grou...,Positive
2,Surely one of the best lasagna you can have. T...,Positive
3,"Closed ambiance, loud. Food was filling, we or...",Positive
4,It was great experience . You will find crowd ...,Positive
